In [2]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 81.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 77.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 23.9 MB/s eta 0:00:00


In [3]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch
from transformers import AdamW
from tqdm import tqdm  # for our progress bar
import torch.nn.functional as F
import numpy as np
import pandas as pd

In [4]:
epochs = 5
ckpt_path="/content/drive/MyDrive/Reserach_Notebooks/ckpt/"
data_path="/content/drive/MyDrive/Reserach_Notebooks/data/"

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
Seq_len=126
train_count=30000

In [5]:
#tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
#model = AutoModelForMaskedLM.from_pretrained('distilbert-base-uncased')

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModelForMaskedLM.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
text = "Messi had a severe [MASK] injury."

inputs = tokenizer(text, return_tensors="pt")
outputs = model(**inputs)
token_logits= outputs.logits.detach().numpy()[0]
# Find the location of [MASK] and extract its logits
mask_token_index = np.where(inputs["input_ids"] == tokenizer.mask_token_id)[1][0]
top_5_tokens= np.argsort(-token_logits[mask_token_index])[:5].tolist()
for token in top_5_tokens:
  print(text.replace('[MASK]', tokenizer.decode(token)))

Messi had a severe head injury.
Messi had a severe back injury.
Messi had a severe knee injury.
Messi had a severe leg injury.
Messi had a severe neck injury.


In [7]:
df = pd.read_csv(data_path+'imdb/IMDB Dataset.csv')
texts=df.review.tolist()

all_texts=''
for t in texts:
  all_texts= all_texts+' '+t

KeyboardInterrupt: ignored

In [8]:
with open(data_path+'imdb/all_texts.txt') as file:
  all_texts=file.read()

all_words=all_texts.split()
train_samples=[]
frm=0
to=Seq_len
for i in range(train_count):
  temp=' '.join(all_words[frm:to])
  train_samples.append(temp)
  frm+=Seq_len
  to+=Seq_len

In [9]:
inputs = tokenizer(train_samples, return_tensors='pt', max_length=128, truncation=True, padding='max_length')

In [10]:
inputs['labels'] = inputs.input_ids.detach().clone()
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [11]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)
mask_arr

tensor([[False, False,  True,  ..., False, False, False],
        [False, False,  True,  ..., False, False, False],
        [False, False,  True,  ..., False, False, False],
        ...,
        [False,  True, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False,  True, False,  ..., False, False, False]])

In [12]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = tokenizer.mask_token_id
inputs.input_ids

tensor([[ 101, 2028,  103,  ..., 2004, 2008,  102],
        [ 101, 2227,  103,  ..., 4930, 2033,  102],
        [ 101, 2041,  103,  ..., 1013, 1028,  102],
        ...,
        [ 101,  103, 2005,  ..., 1998, 2081,  102],
        [ 101, 2008, 2009,  ..., 2027, 2031,  102],
        [ 101,  103, 3492,  ..., 3185, 1037,  102]])

In [13]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

dataset = Dataset(inputs)
loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

In [14]:
# and move our model over to the selected device
model.to(device)
# activate training mode
model.train()

# initialize optimizer
optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

        # to avoid cudaOutOfMemoryError
        torch.cuda.empty_cache()


torch.save(model.state_dict(), ckpt_path+'bert_mlm.pt')

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/938 [00:00<?, ?it/s]<ipython-input-13-95040326d1a9>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Epoch 4: 100%|██████████| 938/938 [13:54<00:00,  1.12it/s, loss=0.176]


In [9]:
#model.load_state_dict(torch.load(ckpt_path+'distilbert_mlm.pt'))
text = "Mark will lead the [MASK] project"

inputs = tokenizer(text, return_tensors="pt")
outputs = model(**inputs)
token_logits= outputs.logits.detach().numpy()[0]
# Find the location of [MASK] and extract its logits
mask_token_index = np.where(inputs["input_ids"] == tokenizer.mask_token_id)[1][0]
top_5_tokens= np.argsort(-token_logits[mask_token_index])[:5].tolist()
for token in top_5_tokens:
  print(text.replace('[MASK]', tokenizer.decode(token)))

Mark will lead the whole project
Mark will lead the entire project
Mark will lead the new project
Mark will lead the next project
Mark will lead the future project
